In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
    
        super(UNet, self).__init__()
        # Double Convolution Block - Conv -> ReLU -> Conv -> ReLU
        def double_conv(in_channels, out_channels):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
                nn.ReLU(inplace=True)
            )
        # Encoder Path (Downsampling)
        self.encoder1 = double_conv(in_channels, 64)    # First encoder block
        self.encoder2 = double_conv(64, 128)             # Second encoder block
        self.encoder3 = double_conv(128, 256)            # Third encoder block
        self.encoder4 = double_conv(256, 512)            # Fourth encoder block
        # MaxPooling layer to downsample
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Bottleneck (bridge between encoder and decoder)
        self.bottleneck = double_conv(512, 1024)
        # Decoder Path (Upsampling + Skip Connections)
        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.decoder4 = double_conv(1024 + 512, 512)    # Concatenate skip connection
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.decoder3 = double_conv(256 + 256, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder2 = double_conv(128 + 128, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder1 = double_conv(64 + 64, 64)
        # Final Convolution Layer - Maps to desired number of output channels (segmentation mask)
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        """
        Forward pass of U-Net.
        Applies encoder, bottleneck, decoder, and final conv to input image.
        """
        # Encoder
        enc1 = self.encoder1(x)             # Downsample 1
        enc2 = self.encoder2(self.pool(enc1)) # Downsample 2
        enc3 = self.encoder3(self.pool(enc2)) # Downsample 3
        enc4 = self.encoder4(self.pool(enc3)) # Downsample 4
        # Bottleneck
        bottleneck = self.bottleneck(self.pool(enc4))
        # Decoder with Skip Connections
        dec4 = self.upconv4(bottleneck)      # Upsample 1
        dec4 = torch.cat((dec4, enc4), dim=1) # Concatenate skip connection
        dec4 = self.decoder4(dec4)
        dec3 = self.upconv3(dec4)             # Upsample 2
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)
        dec2 = self.upconv2(dec3)             # Upsample 3
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)
        dec1 = self.upconv1(dec2)             # Upsample 4
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)
        return self.final_conv(dec1)
